# 상태유지 스택 순환신경망 모델

In [23]:
import pandas as pd
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY=[], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back),0])
        dataY.append(signal_data[i+look_back,0])
    return np.array(dataX), np.array(dataY)

In [25]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss=[]
        self.val_loss=[]
    
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
    

In [26]:
train_ = pd.read_csv('../data/train.csv',index_col=0)
test_ = pd. read_csv('../data/test.csv',index_col=0)
sample_submission = pd.read_csv('../data/sample_submission.csv', index_col=0)

In [52]:
look_back=5


In [53]:
signal_data=train_['Y00'][0:4320,None]

In [54]:
signal_data.shape

(4320, 1)

In [55]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#scaler=MinMaxScaler(feature_range=(0,1))
scaler=StandardScaler()
signal_data=scaler.fit_transform(signal_data)

In [56]:
length=len(signal_data)


In [57]:
len1=int(0.8*length)
len2=int(0.1*length)

In [58]:
train = signal_data[:len1]
val = signal_data[len1:len1+len2]
test = signal_data[len1+len2:]

In [59]:
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

In [60]:
x_train=np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val=np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test=np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [61]:
model= Sequential()
for i in range(3):
    model.add(LSTM(16, batch_input_shape=(1, look_back,1) ,stateful=True, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.1))

model.add(LSTM(16,  batch_input_shape=(1, look_back,1) , stateful=True, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (1, 5, 16)                1152      
_________________________________________________________________
dropout_11 (Dropout)         (1, 5, 16)                0         
_________________________________________________________________
lstm_12 (LSTM)               (1, 5, 16)                2112      
_________________________________________________________________
dropout_12 (Dropout)         (1, 5, 16)                0         
_________________________________________________________________
lstm_13 (LSTM)               (1, 5, 16)                2112      
_________________________________________________________________
dropout_13 (Dropout)         (1, 5, 16)                0         
_________________________________________________________________
lstm_14 (LSTM)               (1, 16)                   2112      
__________

In [62]:
model.compile(loss='MSE',optimizer='adam')
custom_hist=CustomHistory()
custom_hist.init()

In [ ]:
for i in range(120):
    print(i)
    model.fit( x_train, y_train, epochs=1, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))
    model.reset_states()

0
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.1118 - val_loss: 0.0301
1
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.0552 - val_loss: 0.0211
2
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.0392 - val_loss: 0.0308
3
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.0367 - val_loss: 0.0101
4
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.0335 - val_loss: 0.0403
5
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [==============================] - 30s 9ms/step - loss: 0.0331 - val_loss: 0.0154
6
Train on 3451 samples, validate on 427 samples
Epoch 1/1
3451/3451 [========================

In [ ]:
trainScore=model.evaluate(x_train, y_train, verbose=0,batch_size=1)
model.reset_states()
print('Train Score:', trainScore)

ValScore=model.evaluate(x_val, y_val, verbose=0,batch_size=1)
model.reset_states()
print('Validation Score:', ValScore)

testScore=model.evaluate(x_test, y_test, verbose=0,batch_size=1)
model.reset_states()
print('Test Score:', testScore)


In [ ]:
look_ahead=len(train_['Y00'])-len(train_['Y00'][0:4320])

In [ ]:
len(y_test)

In [ ]:
look_ahead=432+look_back

xhat=x_test[0]
predictions=np.zeros((look_ahead,1))

for i in range(look_ahead):
    prediction=model.predict(np.array([xhat]), batch_size=1)
    predictions[i]=prediction
    xhat=np.vstack([xhat[1:,],prediction])

In [ ]:
# plt.figure(figsize=(12,5))
# plt.plot(np.arange(look_ahead),predictions,'r',label='prediction')
# plt.plot(np.arange(look_ahead),y_test[:412],label='test function')
# plt.legend()
# plt.show()


In [ ]:
le1=np.arange(len(predictions))
le2=np.arange(len(y_test))

plt.plot(le1,predictions,'r',label='prediction')
plt.plot(le2,y_test,label='test function')
